# 🏦 Personal Loan Acceptance (k-NN Model in Python)
Predicting loan acceptance using k-Nearest Neighbors on UniversalBank dataset.

In [ ]:
# Step 1: Import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Step 2: Load data and drop non-predictive columns
df = pd.read_csv('UniversalBank.csv')
df = df.drop(['ID', 'ZIP Code'], axis=1)
df['Personal Loan'] = df['Personal Loan'].map({0: 'nonacceptor', 1: 'acceptor'})
df['Personal Loan'] = df['Personal Loan'].astype('category')
df['Education'] = df['Education'].astype('category')
df.head()

In [ ]:
# Step 3: Encode categorical and binary features
df_encoded = pd.get_dummies(df, drop_first=True)
y = df['Personal Loan'].map({'nonacceptor': 0, 'acceptor': 1})
X = df_encoded.drop('Personal Loan_nonacceptor', axis=1)

In [ ]:
# Step 4: Split the data (60% train / 40% holdout)
X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, test_size=0.4, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_holdout_scaled = scaler.transform(X_holdout)

In [ ]:
# Step 5: Train k-NN with k=1 for a single prediction
knn_1 = KNeighborsClassifier(n_neighbors=1)
knn_1.fit(X_train_scaled, y_train)
sample = pd.DataFrame([[40,10,84,2,2,0,0,0,1,1,0,0]],
                      columns=['Age','Experience','Income','Family','CCAvg','Mortgage','Securities Account',
                               'CD Account','Online','CreditCard','Education_2','Education_3'])
sample_scaled = scaler.transform(sample)
prediction = knn_1.predict(sample_scaled)
print('Prediction for customer:', 'acceptor' if prediction[0] == 1 else 'nonacceptor')

In [ ]:
# Step 6: Optimize k using GridSearchCV with 10-fold CV
param_grid = {'n_neighbors': list(range(1, 21))}
grid = GridSearchCV(KNeighborsClassifier(), param_grid, cv=10, scoring='accuracy')
grid.fit(X_train_scaled, y_train)
print('Best k:', grid.best_params_['n_neighbors'])
best_knn = grid.best_estimator_

In [ ]:
# Step 7: Evaluate on holdout set
y_pred = best_knn.predict(X_holdout_scaled)
cm = confusion_matrix(y_holdout, y_pred)
acc = accuracy_score(y_holdout, y_pred)
prec = precision_score(y_holdout, y_pred)
rec = recall_score(y_holdout, y_pred)
print('Confusion Matrix:\n', cm)
print(f'Accuracy: {acc:.2f}, Precision: {prec:.2f}, Recall: {rec:.2f}')